In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import csv
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import math
from matplotlib import pyplot as plt
import collections
import Dataset as ds
import DatasetModifier as dsm
import Classifications as classifications

Using TensorFlow backend.


In [2]:
# Define the parameter creation steps

datasetModifier = dsm.DatasetModifier()
datasetModifier.dataset_randomize()

datasetModifier.dataset_fill_missing_number('Age', 30)
datasetModifier.dataset_categorize_number('Age', [['infant', 0, 2], ['child', 2, 10], ['teenager', 10, 18], ['youngadult', 18, 30], ['midlife', 30, 50], ['oldfart', 50, math.inf]])
datasetModifier.add_X_parameter('Age')
datasetModifier.one_hot_X_parameter('Age')

datasetModifier.add_X_parameter('Sex')
datasetModifier.one_hot_X_parameter('Sex')

datasetModifier.dataset_fill_missing_number('Fare', 0)
datasetModifier.dataset_categorize_number('Fare', [['poor feck', 0, 10], ['middle class', 10, 50], ['richy rich', 50, math.inf]])
datasetModifier.add_X_parameter('Fare')
datasetModifier.one_hot_X_parameter('Fare')

datasetModifier.standardize_X()

datasetModifier.add_Y_parameter('Survived')

In [3]:
# Load the train/test dataset
dataset = ds.Dataset()
dataset.load_dataset_from_csv('train.csv')

In [4]:
dataset.find_null_columns()

Age         177
Cabin       687
Embarked      2
dtype: int64

In [5]:
# Load the prediction dataset
dataset_prediction = ds.Dataset()
dataset_prediction.load_dataset_from_csv('test.csv')

In [6]:
# Apply the parameter creation steps to the two datasets
datasetModifier.generate_dataset(dataset)
datasetModifier.generate_X(dataset)
datasetModifier.generate_Y(dataset)

datasetModifier.generate_dataset(dataset_prediction)
datasetModifier.generate_X(dataset_prediction)

D:\Programs\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
D:\Programs\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a L

In [7]:
train_X, test_X = dataset.get_X_train_test_sets(0.2)
train_Y, test_Y = dataset.get_Y_train_test_sets(0.2)

In [8]:
# Fetch the predict set data
predict_X, _ = dataset_prediction.get_X_train_test_sets(0)
test_passenger_ids = dataset_prediction.get_dataset_parameter('PassengerId')
test_passenger_ids = np.reshape(test_passenger_ids.values, (test_passenger_ids.shape[0], 1))

In [11]:
clf = classifications.Classifications()
#clf.find_svm_linear_params(train_X, train_Y)
#clf.find_svm_kernel_params(train_X, train_Y)
#clf.randomized_random_forest_parameter_search(train_X, train_Y)
#clf.grid_search_for_params(['auto'], [4000, 5000, 5500, 6000, 7000], [5, 8, 10, 15, 20, 50], [4, 5, 6], [1], [True])

In [ ]:
clf.add_logistic_regression(cross_validation=7, rnd_state=0)
clf.add_linear_svm(2.5719138090593443)
clf.add_rbf_svm(8.886238162743407, 0.04124626382901352)
clf.add_naive_bayes()
clf.add_classification_tree(depth=6)
clf.add_random_forest(max_features='auto', max_depth=5, n_estimators=5000, min_samples_split=4, min_samples_leaf=1, bootstrap=True)

In [15]:
best_algorithm = clf.find_best_algorithm(train_X, train_Y, test_X, test_Y)

D:\Programs\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic Regression scored 0.7808988764044944
Linear SVM scored 0.7808988764044944
RBF SVM scored 0.7808988764044944
Naive Bayes scored 0.7921348314606742
Classification Tree scored 0.7752808988764045


D:\Programs\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Programs\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Programs\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Projects\git\Kaggle\Titanic Machine Learning from Disaster\Classifications.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), f

Random Forest scored 0.7752808988764045
Best Model is: Naive Bayes with a score of 0.7921348314606742 - will continue with this


# Predict Model

In [53]:
# Run the model against the test data
predict_Y = best_model.predict(predict_X)
predict_Y = np.around(predict_Y)
predict_Y = predict_Y.astype(np.integer)
predict_Y = np.reshape(predict_Y, (predict_Y.shape[0], 1))

In [54]:
csv_predict = np.concatenate((test_passenger_ids, predict_Y), axis=1)
csv_predict = np.concatenate((np.reshape(["PassengerId", "Survived"], (1, 2)), csv_predict))
with open('prediction.csv', 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csv_predict)
csvFile.close()